# MetPy Collection of Atmospheric Maps from Current GFS Data

## Justin Richling
## 09/20/18

In [ ]:
%matplotlib inline


# Model Surface Output


Plot an surface map with mean sea level pressure (MSLP),
2m Temperature (F), and Wind Barbs (kt).

In this part of the final notebook I had to really dig into the 2m plot data, because it was originally setup for NARR data which had various discrepencies from the GFS NetCDF files. I successfully gotten python to plot the GFS data for the original NARR variables, which was another bit of customization



Imports



In [ ]:
# Random Library Imports
import subprocess,os,glob,tempfile,re,imageio,webbrowser,io,sys,types,urllib,urllib2,\
time,cStringIO

# Importing Datetime Libraries
from datetime import datetime

# CartoPy Map Plotting Libraires
import cartopy.crs as ccrs
import cartopy.feature as cfeature
from pyproj import Proj 

# Numerical and Scientific Libraries
import numpy as np
import scipy.ndimage as ndimage
from scipy.ndimage import gaussian_filter

# Accessing Data from External Databases via XLM Catalog
from siphon.ncss import NCSS
from siphon.catalog import TDSCatalog

# MetPy Libraries
import metpy
import metpy.calc as mpcalc
from metpy.units import units
from metpy.plots import ctables
from metpy.plots import add_metpy_logo

# NetCDF Libraries
from netCDF4 import Dataset
from netCDF4 import num2date

# More Image Manipulation Options
from PIL import Image as PILImage

# Ipyhton Options
from IPython import get_ipython
from nbformat import current
from IPython.core.interactiveshell import InteractiveShell
from IPython.display import HTML, display, Image

# Matplotlib Plotting Libraries
#import matplotlib
import matplotlib.pyplot as plt
from matplotlib.cm import get_cmap
import matplotlib.colors as mcolors
from matplotlib.colors import LogNorm, Normalize
import matplotlib as mpl
from mpl_toolkits.axes_grid1 import make_axes_locatable, axes_size
from matplotlib.colors import LinearSegmentedColormap

## Figuring out where you want the files...

In [ ]:
# Pull the current time
now = datetime.now() 

# Set a path to save the plots with string format for the date to set the month and day 
im_save_path ="/path/to/where/you/want/maps/"+'{0:%m_%d}'.format(now)+"/"
print im_save_path

# Check to see if the folder already exists, if not create it
if not os.path.isdir(im_save_path):
    os.makedirs(im_save_path)

# Uncomment if you want to automatically change to the map folder    
#os.chdir(im_save_path)

# Additional Functions

### Finding proper time variables: find_time_var()

In [ ]:
# Thanks to the crew over at Metpy for this handy little function
def find_time_var(var, time_basename='time'):
    for coord_name in var.coordinates.split():
        if coord_name.startswith(time_basename):
            return coord_name
    raise ValueError('No time variable found for ' + var.name)

### Maximum and Minimum function to locate Highs and Lows: plot_maxmin_points()

In [ ]:
# Again, thanks to MetPy for doing the leg work to get a H/L function
def plot_maxmin_points(lon, lat, data, extrema, nsize, symbol, color='k',
                       plotValue=True, transform=None):
    """
    This function will find and plot relative maximum and minimum for a 2D grid. The function
    can be used to plot an H for maximum values (e.g., High pressure) and an L for minimum
    values (e.g., low pressue). It is best to used filetered data to obtain  a synoptic scale
    max/min value. The symbol text can be set to a string value and optionally the color of the
    symbol and any plotted value can be set with the parameter color
    lon = plotting longitude values (2D)
    lat = plotting latitude values (2D)
    data = 2D data that you wish to plot the max/min symbol placement
    extrema = Either a value of max for Maximum Values or min for Minimum Values
    nsize = Size of the grid box to filter the max and min values to plot a reasonable number
    symbol = String to be placed at location of max/min value
    color = String matplotlib colorname to plot the symbol (and numerica value, if plotted)
    plot_value = Boolean (True/False) of whether to plot the numeric value of max/min point
    The max/min symbol will be plotted on the current axes within the bounding frame
    (e.g., clip_on=True)
    """
    from scipy.ndimage.filters import maximum_filter, minimum_filter

    if (extrema == 'max'):
        data_ext = maximum_filter(data, nsize, mode='nearest')
    elif (extrema == 'min'):
        data_ext = minimum_filter(data, nsize, mode='nearest')
    else:
        raise ValueError('Value for hilo must be either max or min')

    mxy, mxx = np.where(data_ext == data)

    for i in range(len(mxy)):
        ax.text(lon[mxx[i]], lat[mxy[i]], symbol, color=color, size=24,
                clip_on=True, horizontalalignment='center', verticalalignment='center',
                transform=transform)
        ax.text(lon[mxx[i]], lat[mxy[i]],
                '\n' + str(np.int(data[mxy[i], mxx[i]])),
                color=color, size=12, clip_on=True, fontweight='bold',
                horizontalalignment='center', verticalalignment='top', transform=transform)

# Create NCSS object to access the NetcdfSubset
---------------------------------------------
### Data from NCEI GFS 0.5 deg Analysis Archive



In [ ]:
cat = TDSCatalog('http://thredds.ucar.edu/thredds/catalog/grib/'
                 'NCEP/GFS/Global_0p5deg/catalog.xml')
best = cat.datasets['Best GFS Half Degree Forecast Time Series']

# Summary: NCEP GFS Model : AWIPS 230 (G) Grid. Global Lat/Lon grid. Model runs at 0, 6, 12, and 18Z. 
# Horizontal= 361 by 720 points, resolution 0.5 degree, Lat/Lon projection. 
# Vertical= 1000 to 100 hPa mandatory pressure levels (10 levels); surface, height above ground, pressure layers.

subset_access = best.subset()
query = subset_access.query()
#for i in subset_access.variables: print i

query.time(datetime.utcnow())
print datetime.utcnow()
Time = str(datetime.utcnow())[:-7]
print Time
query.variables('Temperature_isobaric', 'Geopotential_height_isobaric',
                'u-component_of_wind_isobaric', 'v-component_of_wind_isobaric',
                'Relative_humidity_isobaric','Absolute_vorticity_isobaric',
                "Vertical_velocity_pressure_isobaric","Pressure_reduced_to_MSL_msl","Pressure_surface",
               'Apparent_temperature_height_above_ground','u-component_of_wind_height_above_ground',
               'v-component_of_wind_height_above_ground','Dewpoint_temperature_height_above_ground')

variables = sorted(v for v in subset_access.variables if v.endswith('temperature'))
#print variables

query.lonlat_box(west=-130, east=-50, south=10, north=60)
query.accept('netcdf4')

data = subset_access.get_data(query)

print "Variables:"
for i in data.variables.keys(): print i,"\n"

print "done."

# Begin data maipulation
-----------------------
Data for the surface from a model is a bit complicated.
The variables come from different levels and may have different data array shapes.

In [ ]:
dew = units.K * data.variables['Dewpoint_temperature_height_above_ground'][:].squeeze()

## MSLP, 2m Temp, and 10m Wind

Height above ground Temp from GFS has one level (2m)
Height above ground Wind from GFS has three levels (10m, 80m, 100m)


In [ ]:
# Pull out variables you want to use
mslp = data.variables['Pressure_reduced_to_MSL_msl'][:].squeeze()
temp = units.K * data.variables['Apparent_temperature_height_above_ground'][:].squeeze()
u_wind = units('m/s') * data.variables['u-component_of_wind_height_above_ground'][:].squeeze()
v_wind = units('m/s') * data.variables['v-component_of_wind_height_above_ground'][:].squeeze()
lat = data.variables['lat'][:].squeeze()
lon = data.variables['lon'][:].squeeze()
lats = data.variables['lat'][:]
lons = data.variables['lon'][:]
print lons.shape,lats.shape
time_var = data.variables[find_time_var(data.variables['Pressure_reduced_to_MSL_msl'])]

# Convert winds to knots
u_wind.ito('kt')
v_wind.ito('kt')

# Convert number of hours since the reference time into an actual date
time_final = num2date(time_var[:].squeeze(), time_var.units)
print str(time_final)[:4]+"_"+str(time_final)[5:7]+"_"+str(time_final)[8:10]+"_"+str(time_final)[11:13]+"Z"
file_time = str(time_final)[:4]+"_"+str(time_final)[5:7]+"_"+str(time_final)[8:10]+"_"+str(time_final)[11:13]+"Z"

lev_10m = np.where(data.variables['height_above_ground3'][:] == 10)[0][0]
u_wind_10m = u_wind[lev_10m]
v_wind_10m = v_wind[lev_10m]

# Combine 1D latitude and longitudes into a 2D grid of locations
lon_2d, lat_2d = np.meshgrid(lon, lat)

# Smooth MSLP a little
# Be sure to only put in a 2D lat/lon or Y/X array for smoothing
smooth_mslp = ndimage.gaussian_filter(mslp, sigma=3, order=0) * units.Pa
smooth_mslp.ito('hPa')


## High/Lows and Thickness Data

In [ ]:
# Grab pressure levels
plev = list(data.variables['isobaric'][:])
print plev

# Grab lat/lons and make all lons 0-360
lats = data.variables['lat'][:]
lons = data.variables['lon'][:]
print lats.shape,lons.shape
lons[lons < 0] = 360 + lons[lons < 0]

# Grab MSLP and smooth, use MetPy Units module for conversion
EMSL = data.variables['Pressure_reduced_to_MSL_msl'][:] * units.Pa
EMSL.ito('hPa')
mslp = gaussian_filter(EMSL[0], sigma=3.0)

# Grab pressure level data
hght_1000 = data.variables['Geopotential_height_isobaric'][0, plev.index(100000.0)]
hght_500 = data.variables['Geopotential_height_isobaric'][0, plev.index(50000.0)]
#mb500 = data.variables['Geopotential_height_isobaric'][0,::plev.index(50000.0)]*units.Pa
#mb500.ito('hPa')
Height500mb = gaussian_filter(hght_500[0], sigma=3.0)

# Calculate and smooth 1000-500 hPa thickness
thickness_1000_500 = gaussian_filter(hght_500 - hght_1000, sigma=3.0)

In [ ]:
lev_500 = np.where(data.variables['isobaric'][:] == 50000)[0][0]
hght_500 = data.variables['Geopotential_height_isobaric'][0, lev_500, :, :]

In [ ]:
lev_250 = np.where(data.variables['isobaric'][:] == 25000)[0][0]

hght_250 = data.variables['Geopotential_height_isobaric'][0, lev_250, :, :]
u_250 = data.variables['u-component_of_wind_isobaric'][0, lev_250, :, :]
v_250 = data.variables['v-component_of_wind_isobaric'][0, lev_250, :, :]

## PV Analysis

In [ ]:
lat = data.variables['lat'][:]
lon = data.variables['lon'][:]
press = data.variables['isobaric'][:] * units.Pa
temperature = data.variables['Temperature_isobaric'][0] * units.kelvin
rh = data.variables['Relative_humidity_isobaric'][0] * units.percent
height = data.variables['Geopotential_height_isobaric'][0] * units.meter
u = data.variables['u-component_of_wind_isobaric'][0] * units('m/s')
v = data.variables['v-component_of_wind_isobaric'][0] * units('m/s')
vort =  data.variables['Absolute_vorticity_isobaric'][0] * units('1/s')

Next, we perform the isentropic interpolation. At a minimum, this must be given one or more isentropic levels, the 3-D temperature field, and the pressure levels of the original field; it then returns the 3D array of pressure values (2D slices for each isentropic level). You can also pass addition fields which will be interpolated to these levels as well. Below, we interpolate the winds (and pressure) to the 295K isentropic level:

In [ ]:
TEMP = 295
isen_level = np.array([TEMP]) * units.kelvin
isen_press, isen_u, isen_v = mpcalc.isentropic_interpolation(isen_level, press,
                                                             temperature, u, v)

In [ ]:
# Need to squeeze() out the size-1 dimension for the isentropic level
isen_press = isen_press.squeeze()
isen_u = isen_u.squeeze()
isen_v = isen_v.squeeze()
#isen_vort = isen_vort.squeeze()

In [ ]:
MIN = data.variables['Absolute_vorticity_isobaric'][0,15,:,:].min()
MAX = data.variables['Absolute_vorticity_isobaric'][0,15,:,:].max()

In [ ]:
# Needed to make numpy broadcasting work between 1D pressure and other 3D arrays
pressure_for_calc = press[:, None, None]

# Calculate mixing ratio using something from mpcalc
mixing = mpcalc.mixing_ratio_from_relative_humidity(rh, temperature, pressure_for_calc)

# Take the return and convert manually to units of 'dimenionless'
mixing.ito('dimensionless')

# Interpolate all the data
isen_level = np.array([295]) * units.kelvin
ret = mpcalc.isentropic_interpolation(isen_level, press, temperature, mixing, u, v)
isen_press, isen_mixing, isen_u, isen_v = ret

# Squeeze the returned arrays
isen_press = isen_press.squeeze()
isen_mixing = isen_mixing.squeeze()
isen_u = isen_u.squeeze()
isen_v = isen_v.squeeze()


<a name="ascent"></a>
## Calculating Isentropic Ascent

Air flow across isobars on an isentropic surface represents vertical motion. We can use MetPy to calculate this ascent for us.

Since calculating this involves taking derivatives, first let's smooth the input fields using the `gaussian_filter` from `scipy.ndimage`. Unfortunately, `gaussian_filter` drops units from the input vluaes, so we should see what units we have so we can reattach afterwards. (In the future, MetPy will provide its own version of `gaussian_filter` to avoid this.)

In [ ]:
isen_press.units, isen_u.units, isen_v.units

In [ ]:
dx, dy = mpcalc.lat_lon_grid_spacing(lon, lat)
dy = -dy

Begin map creation
------------------



In [ ]:
# Set Projection of Data
datacrs = ccrs.PlateCarree()

# Set Projection of Plot
plotcrs = ccrs.LambertConformal(central_latitude=[30, 60], central_longitude=-100)

# Set the state/provinces
states_provinces = cfeature.NaturalEarthFeature(category='cultural',
    name='admin_1_states_provinces_lakes',scale='50m', facecolor='none')

# Set the country borders
country_borders = cfeature.NaturalEarthFeature(category='cultural',
    name='admin_0_countries',scale='50m', facecolor='none')

# Colorbar Axis Placement (under figure)
colorbar_axis = [0.183, 0.09, 0.659, 0.03] # [left, bottom, width, height]

# Lat/Lon Extents [lon_west, lon_east, lat_south, lat_north]
extent = [-130., -70, 20., 60.]

# Plots

## MSLP, 2m Temps, and 10m Winds

In [ ]:
# Create new figure
fig = plt.figure(figsize=(17., 11.))

add_metpy_logo(fig, 30, 940, size='small')

# Add the map and set the extent
ax = plt.subplot(111, projection=plotcrs)

#Set the lat and lon boundaries
ax.set_extent(extent, datacrs)

# Add state boundaries to plot
ax.add_feature(states_provinces, edgecolor='blue', linewidth=1)

# Add country borders to plot
ax.add_feature(country_borders, edgecolor='black', linewidth=1)

# Plot Title
plt.title('GFS Analysis MSLP, 2m Temperature (F), Wind Barbs (kt)', fontsize=16,loc='left')
plt.title(' {0:%d %B %Y %H:%MZ}'.format(time_final), fontsize=16,loc='right')

                                            # MSLP
#---------------------------------------------------------------------------------------------------
#clev_mslp = np.arange(0, 1200, 3)

#cs = ax.contour(lon_2d, lat_2d, smooth_mslp, clev_mslp, colors='k', linewidths=3,
#    linestyles='solid', transform=datacrs) # cmap='rainbow

#plt.clabel(cs, fontsize=10, inline=1, inline_spacing=10, fmt='%i',
#           rightside_up=True, use_clabeltext=True,colors='k')

# Plot Absolute Vorticity
#clev_abVort = np.arange(Max,Min,0.00001)
#abVort = ax.contourf(lon_2d, lat_2d, abs_vort[5,:,:],transform=datacrs,cmap='BrBG')
#cbaxes = fig.add_axes([0.92, 0.233, 0.03, 0.54]) # [left, bottom, width, height]
#cbar = plt.colorbar(abVort, orientation='vertical',cax=cbaxes)
#plt.colorbar(cs2, orientation='vertical', fraction=0.0311, pad=0.02)


                                            # 2m Temperatures
#---------------------------------------------------------------------------------------------------
# Plot 2m Temperature Contours
clevtemp = np.arange(-30, 90, 2)

# Uncomment for contours instead of contour fill (below)
#cs2 = ax.contour(lon_2d, lat_2d, temp.to(units('degF')), clevtemp,
#                 cmap='gist_rainbow_r', linewidths=1.25,
#                 transform=datacrs)  #linestyles='dotted'
#plt.clabel(cs2, fontsize=10, inline=1, inline_spacing=10, fmt='%i',
#           rightside_up=True, use_clabeltext=True)

cs2 = ax.contourf(lon_2d, lat_2d, temp.to(units('degF')),clevtemp,
                 transform=datacrs,cmap='gist_rainbow_r')

cbaxes = fig.add_axes(colorbar_axis) # [left, bottom, width, height]

cbar = plt.colorbar(cs2, orientation='horizontal',cax=cbaxes)


                                            # 10m Winds
#---------------------------------------------------------------------------------------------------
ax.barbs(lon_2d, lat_2d, u_wind_10m.magnitude, v_wind_10m.magnitude,
         length=6, regrid_shape=20, pivot='middle', transform=datacrs,barbcolor='k')

plt.show()

outfile=im_save_path+"MSLP_Temps_Winds_"+file_time+".png"
fig.savefig(outfile,bbox_inches='tight',dpi=120)
print "done."

## MSLP w/ Highs and Lows and 1000-500mn Thickness

In [ ]:
# Create new figure
fig = plt.figure(figsize=(17., 11.))

add_metpy_logo(fig, 30, 1000, size='small')

# Add the map 
ax = plt.subplot(111, projection=plotcrs)

# Set extent and plot map lines
ax.set_extent(extent, datacrs)
#ax.set_extent([-75, -30.5, 0.5, 28.5])

# Add state boundaries to plot
ax.add_feature(states_provinces, edgecolor='k', linewidth=1)

# Add country borders to plot
ax.add_feature(country_borders, edgecolor='black', linewidth=1)

# Plot Title
plt.title('MSLP (hPa) with Highs and Lows, 1000-500 hPa Thickness (m)', loc='left',fontsize=16)
plt.title(' {0:%d %B %Y %H:%MZ}'.format(time_final), loc='right',fontsize=16)

                                            # 1000-500mb Thickness
#---------------------------------------------------------------------------------------------------
# Setting the limits for the dashed thickness lines to go from blue to red
clevs = (np.arange(0, 5400, 60), 5400, np.arange(5460, 7000, 60))
colors = ('tab:blue', 'b', 'tab:red')
kw_clabels = {'fontsize': 11, 'inline': True, 'inline_spacing': 5, 'fmt': '%i',
              'rightside_up': True, 'use_clabeltext': True}
for clevthick, color in zip(clevs, colors):
    cs = ax.contour(lons, lats, thickness_1000_500, levels=clevthick, colors=color,
                    linewidths=1.0, linestyles='dashed', transform=datacrs)
    plt.clabel(cs, **kw_clabels)

                                            # MSLP
#---------------------------------------------------------------------------------------------------
clevmslp = np.arange(800., 1120., 4)
cs2 = ax.contour(lons, lats, mslp, clevmslp, colors='k', linewidths=1.25,
                 linestyles='solid', transform=datacrs)
plt.clabel(cs2, **kw_clabels)

                                            # High and Low Symbols
#---------------------------------------------------------------------------------------------------
plot_maxmin_points(lons, lats, mslp, 'max', 50, symbol='H', color='b',  transform=datacrs)
plot_maxmin_points(lons, lats, mslp, 'min', 25, symbol='L', color='r', transform=datacrs)

plt.show()

outfile=im_save_path+"HL_Thickness_Model_"+file_time+".png"
fig.savefig(outfile,bbox_inches='tight',dpi=120)
print "done."

## Isentropic Pressure and Winds

In [ ]:
# Create a plot and basic map projection
fig = plt.figure(figsize=(17., 11.))

add_metpy_logo(fig, 30, 940, size='small')

# Add the map and set the extent
ax = plt.subplot(111, projection=plotcrs)

# Add state boundaries to plot
ax.add_feature(states_provinces, edgecolor='k', linewidth=1)

# Add country borders to plot
ax.add_feature(country_borders, edgecolor='black', linewidth=1)

# Plot Title
plt.title('{}K Isentrope Heights and Winds'.format(TEMP),loc='left',fontsize=16)
plt.title(' {0:%d %B %Y %H:%MZ}'.format(time_final),loc='right',fontsize=16)

                                            # Isentropic Pressure
#---------------------------------------------------------------------------------------------------
# Contour the pressure values for the isentropic level. We keep the handle
# for the contour so that we can have matplotlib label the contours

levels = np.arange(300, 1000, 10)
cntr = ax.contour(lon, lat, isen_press, transform=datacrs,
                 cmap='rainbow',levels=levels,linewidths=2) #ccrs.PlateCarree()
#ax.clabel(cntr, fmt='%.0f',colors='black')

cbaxes = fig.add_axes(colorbar_axis)
cbar = plt.colorbar(cntr, orientation='horizontal',cax=cbaxes)

                                            # Isentropic Winds
#---------------------------------------------------------------------------------------------------
# Set up slices to subset the wind barbs--the slices below are the same as `::5`
# We put these here so that it's easy to change and keep all of the ones below matched
# up.

lon_slice = slice(None, None, 7)
lat_slice = slice(None, None, 7)
ax.barbs(lon[lon_slice], lat[lat_slice],
         isen_u[lon_slice, lat_slice].to('knots').magnitude,
         isen_v[lon_slice, lat_slice].to('knots').magnitude,
         transform=ccrs.PlateCarree(), zorder=2,length=7) # barbcolor="" optional call

ax.set_extent(extent, datacrs)

plt.show()

outfile=im_save_path+"Isentropes_Winds_"+file_time+".png"
fig.savefig(outfile,bbox_inches='tight',dpi=120)
print "done."

## Isentropic Pressure, Winds and Vorticity

In [ ]:
# Create a plot and basic map projection
fig = plt.figure(figsize=(17., 11.))

add_metpy_logo(fig, 30, 940, size='small')

# Add the map and set the extent
ax = plt.subplot(111, projection=plotcrs)

# Add state boundaries to plot
ax.add_feature(states_provinces, edgecolor='k', linewidth=1)

# Add country borders to plot
ax.add_feature(country_borders, edgecolor='black', linewidth=1)

# Plot Title
plt.title('Vorticity and Winds',loc='left',fontsize=16)
plt.title(' {0:%d %B %Y %H:%MZ}'.format(time_final),loc='right',fontsize=16)

                                            # Isentropic Pressure
#---------------------------------------------------------------------------------------------------
#levels = np.arange(300, 1000, 50)
#cntr = ax.contour(lon, lat, isen_press, transform=ccrs.PlateCarree(),
#                  colors='red', levels=levels)
#cntr = ax.contour(lon, lat, isen_press, transform=ccrs.PlateCarree(),
#                  cmap='rainbow',levels=levels)
#ax.clabel(cntr, fmt='%.0f',colors='black')

#cbaxes = fig.add_axes([0.82, 0.133, 0.02, 0.74]) # [left, bottom, width, height]
#cbar = plt.colorbar(cntr, orientation='vertical',cax=cbaxes)

                                            # Vorticity
#---------------------------------------------------------------------------------------------------
# draw filled contours.
clevs = [0, 1, 2.5, 5, 7.5, 10, 15, 20, 30, 40,
         50, 70, 100, 150, 200, 250, 300, 400, 500, 600, 750]
# In future MetPy
#norm, cmap = ctables.registry.get_with_boundaries('precipitation', clevs)
cmap_data = [(1.0, 1.0, 1.0),
             (0.3137255012989044, 0.8156862854957581, 0.8156862854957581),
             (0.0, 1.0, 1.0),
             (0.0, 0.8784313797950745, 0.501960813999176),
             (0.0, 0.7529411911964417, 0.0),
             (0.501960813999176, 0.8784313797950745, 0.0),
             (1.0, 1.0, 0.0),
             (1.0, 0.6274510025978088, 0.0),
             (1.0, 0.0, 0.0),
             (1.0, 0.125490203499794, 0.501960813999176),
             (0.9411764740943909, 0.250980406999588, 1.0),
             (0.501960813999176, 0.125490203499794, 1.0),
             (0.250980406999588, 0.250980406999588, 1.0),
             (0.125490203499794, 0.125490203499794, 0.501960813999176),
             (0.125490203499794, 0.125490203499794, 0.125490203499794),
             (0.501960813999176, 0.501960813999176, 0.501960813999176),
             (0.8784313797950745, 0.8784313797950745, 0.8784313797950745),
             (0.9333333373069763, 0.8313725590705872, 0.7372549176216125),
             (0.8549019694328308, 0.6509804129600525, 0.47058823704719543),
             (0.6274510025978088, 0.42352941632270813, 0.23529411852359772),
             (0.4000000059604645, 0.20000000298023224, 0.0)]
cmap = mcolors.ListedColormap(cmap_data, 'precipitation')
norm = mcolors.BoundaryNorm(clevs, cmap.N)

colors = [(1, 0, 0), (1, 1, 1), (0, 0, 1)]
n_bin = 10  # Discretizes the interpolation into bins
cmap_name = 'my_list'

levels = np.arange(MIN,MAX,.00001)

# Create the colormap
cm = LinearSegmentedColormap.from_list("seismic",colors, N=n_bin)
cntr = ax.contourf(lon, lat, vort[15,:,:], transform=datacrs,
                  cmap="Spectral_r",levels=levels)  # cmap="Spectral_r"

cbaxes = fig.add_axes(colorbar_axis) # [left, bottom, width, height]
cbar = plt.colorbar(cntr, orientation='horizontal',cax=cbaxes)

                                            # Isentropic Winds
#---------------------------------------------------------------------------------------------------
# Set up slices to subset the wind barbs--the slices below are the same as `::5`
# We put these here so that it's easy to change and keep all of the ones below matched
# up.
lon_slice = slice(None, None, 5)
lat_slice = slice(None, None, 5)
ax.barbs(lon[lon_slice], lat[lat_slice],
         isen_u[lon_slice, lat_slice].to('knots').magnitude,
         isen_v[lon_slice, lat_slice].to('knots').magnitude,
         transform=ccrs.PlateCarree(), zorder=2) # barbcolor="" optional call

ax.set_extent(extent, datacrs)

plt.show()
outfile=im_save_path+"Isentropes_Vorticity_Winds_"+file_time+".png"
fig.savefig(outfile,bbox_inches='tight',dpi=120)
print "done."

## Isentropic Pressure, Winds and Mixing Ratio

In [ ]:
# Create a plot and basic map projection
fig = plt.figure(figsize=(17., 11.))

add_metpy_logo(fig, 30, 940, size='small')

# Add the map and set the extent
ax = plt.subplot(111, projection=plotcrs)

# Add state boundaries to plot
ax.add_feature(states_provinces, edgecolor='k', linewidth=1)

# Add country borders to plot
ax.add_feature(country_borders, edgecolor='black', linewidth=1)

# Plot Title
plt.title('{}K Isentrope Pressure, Winds and Mixing Ratio'.format(TEMP),loc='left',fontsize=16)
plt.title(' {0:%d %B %Y %H:%MZ}'.format(time_final),loc='right',fontsize=16)

                                            # Isentropic Pressure
#---------------------------------------------------------------------------------------------------
levels = np.arange(300, 1000, 20)
cntr = ax.contour(lon, lat, isen_press, transform=datacrs,
                  cmap='rainbow', levels=levels,linewidths=2.0)#colors='black'
ax.clabel(cntr, fmt='%.0f')

                                            # Isentropic Winds
#---------------------------------------------------------------------------------------------------
lon_slice = slice(None, None, 8)
lat_slice = slice(None, None, 8)
ax.barbs(lon[lon_slice], lat[lat_slice],
         isen_u[lon_slice, lat_slice].to('knots').magnitude,
         isen_v[lon_slice, lat_slice].to('knots').magnitude,
         transform=ccrs.PlateCarree(), zorder=3,barbcolor='k')


                                           # Isentropic Mixing Ratio
#---------------------------------------------------------------------------------------------------
# Contourf the mixing ratio values
mixing_levels = [0.001, 0.002, 0.004, 0.006, 0.010, 0.012, 0.014, 0.016, 0.020]
cntr2 = ax.contourf(lon, lat, isen_mixing, transform=datacrs,
            levels=mixing_levels, cmap='YlGn')

ax.set_extent(extent, datacrs)

cbaxes = fig.add_axes(colorbar_axis) # [left, bottom, width, height]
cbar = plt.colorbar(cntr2, orientation='horizontal',cax=cbaxes)

plt.show()

outfile=im_save_path+"Isentropes_MixingRatio_Winds_"+file_time+".png"
fig.savefig(outfile,bbox_inches='tight',dpi=120)
print "done."

### We can smooth the data, why I'm not sure...

In [ ]:
# Filter and re-attach units
isen_press = gaussian_filter(isen_press.squeeze(), sigma=2.0) * units.hPa
isen_u = gaussian_filter(isen_u.squeeze(), sigma=2.0) * units('m/s')
isen_v = gaussian_filter(isen_v.squeeze(), sigma=2.0) * units('m/s')
lift = -mpcalc.advection(isen_press, [isen_u, isen_v], [dx, dy], dim_order='yx')

## Isentropic Pressure, Winds and Omega

In [ ]:
# Create a plot and basic map projection
fig = plt.figure(figsize=(17., 11.))

add_metpy_logo(fig, 30, 940, size='small')

# Add the map and set the extent
ax = plt.subplot(111, projection=plotcrs)

# Add state boundaries to plot
ax.add_feature(states_provinces, edgecolor='k', linewidth=1)

# Add country borders to plot
ax.add_feature(country_borders, edgecolor='black', linewidth=1)

# Plot Title
plt.title('{}K Isentropes, Winds and Omega'.format(TEMP),loc='left',fontsize=16)
plt.title(' {0:%d %B %Y %H:%MZ}'.format(time_final),loc='right',fontsize=16)

                                            # Isentropic Pressure
#---------------------------------------------------------------------------------------------------
levels = np.arange(300, 1000, 50)
cntr = ax.contour(lon, lat, isen_press, transform=ccrs.PlateCarree(), colors='black', levels=levels)
ax.clabel(cntr, fmt='%.0f')


                                            # Isentropic Winds
#---------------------------------------------------------------------------------------------------
lon_slice = slice(None, None, 7)
lat_slice = slice(None, None, 7)
ax.barbs(lon[lon_slice], lat[lat_slice],
         isen_u[lon_slice, lat_slice].to('knots').magnitude,
         isen_v[lon_slice, lat_slice].to('knots').magnitude,
         transform=ccrs.PlateCarree(), zorder=2)


                                            # Omega
#---------------------------------------------------------------------------------------------------
levels = np.arange(-10, 10,2)
cs = ax.contourf(lon, lat, lift.to('microbar/s'), levels=levels, cmap='RdBu',
                 transform=ccrs.PlateCarree())#, extend='both')
cbaxes = fig.add_axes(colorbar_axis) 
cbar = plt.colorbar(cs, orientation='horizontal',cax=cbaxes)

ax.set_extent(extent, datacrs)

plt.show(fig)
outfile=im_save_path+"Isentropes_Omega_Winds_"+file_time+".png"
fig.savefig(outfile,bbox_inches='tight',dpi=120)
print "done."

## 1000-500mb Heights

In [ ]:
# Create a figure object, title it, and do the plots.
fig = plt.figure(figsize = (17.,11.))

add_metpy_logo(fig, 30, 940, size='small')

# Add the map and set the extent
ax5 = plt.subplot(1,1,1, projection=plotcrs)
ax5.set_extent(extent, datacrs)

# Add state boundaries to plot
ax5.add_feature(states_provinces, edgecolor='k', linewidth=1)

# Add country borders to plot
ax5.add_feature(country_borders, edgecolor='black', linewidth=1)

# Plot Title
plt.title('1000 and 500mb Heights',loc='left',fontsize=16)
plt.title(' {0:%d %B %Y %H:%MZ}'.format(time_final),loc='right',fontsize=16)

                                        # Heights - 1000mb
#---------------------------------------------------------------------------------------------------
lev_1000 = np.where(data.variables['isobaric'][:] == 100000)[0][0]

hght_1000 = data.variables['Geopotential_height_isobaric'][0, lev_1000, :, :]
MIN = hght_1000.min()
MAX = hght_1000.max()
print hght_1000.min(),hght_1000.max()
hght_1000 = ndimage.gaussian_filter(hght_1000, sigma=3, order=0) * units.meter

clev1000 = np.arange(MIN, MAX, 50)
cs = ax5.contour(lon, lat, hght_1000.m, clev1000, colors='blue', linewidths=2.0,
                linestyles='solid', transform=ccrs.PlateCarree())
plt.clabel(cs, fontsize=10, inline=1, inline_spacing=10, fmt='%i',
           rightside_up=True, use_clabeltext=True)

                                        # Heights - 500mb
#---------------------------------------------------------------------------------------------------
MIN = hght_500.min()
MAX = hght_500.max()
print hght_500.min(),hght_500.max()
hght_500 = ndimage.gaussian_filter(hght_500, sigma=3, order=0) * units.meter

clev500 = np.arange(MIN, MAX, 70)
cs = ax5.contour(lon, lat, hght_500.m, clev500, colors='black', linewidths=2.0,
                linestyles='solid', transform=ccrs.PlateCarree())
plt.clabel(cs, fontsize=10, inline=1, inline_spacing=10, fmt='%i',
           rightside_up=True, use_clabeltext=True)


                                        # Vorticity
#---------------------------------------------------------------------------------------------------
#uwnd_500 = data.variables['u-component_of_wind_isobaric'][0, lev_500, :, :] * units('m/s')
#vwnd_500 = data.variables['v-component_of_wind_isobaric'][0, lev_500, :, :] * units('m/s')
#dx, dy = mpcalc.lat_lon_grid_spacing(lon, lat)#

#lat = data.variables['lat'][:]
#f = mpcalc.coriolis_parameter(np.deg2rad(lat)).to(units('1/sec'))

#avor = mpcalc.vorticity(uwnd_500, vwnd_500, dx, dy, dim_order='yx') + f
#print avor.units
#MAX = avor2.max()
#MIN = avor2.min()
#print MAX*units('dimensionless'),MIN*units('dimensionless')

#avor2 = ndimage.gaussian_filter(avor, sigma=3, order=0) * units('1/s')


#vort_adv = mpcalc.advection(avor2, [uwnd_500, vwnd_500], (dx, dy), dim_order='yx') * 1e9
# Plot Colorfill of Vorticity Advection
#clev_avoradv = np.arange(MIN, MAX, 5)
#cf = ax5.contourf(lon, lat, vort_adv.m, clev_avoradv[clev_avoradv != 0], extend='both',
#                 cmap='bwr', transform=ccrs.PlateCarree())
#cax = plt.subplot(gs[1])
#cb = plt.colorbar(cf, cax=cax, orientation='horizontal', extendrect='True', ticks=clev_avoradv)
#cb.set_label(r'$1/s^2$', size='large')

#---------------------------------------------------------------------------------------------------
#---------------------------------------------------------------------------------------------------

fig.subplots_adjust(top=0.89)
fig.savefig(im_save_path+"1000_500mb_Heights_"+file_time+".png",
            bbox_inches='tight',dpi=120)
print 'done.'

## 250mb Heights

In [ ]:
# Create a figure object, title it, and do the plots.
fig = plt.figure(figsize = (17.,11.))

add_metpy_logo(fig, 30, 1000, size='small')

# Add the map and set the extent
ax6 = plt.subplot(1,1,1, projection=plotcrs)

# Add state boundaries to plot
ax6.add_feature(states_provinces, edgecolor='b', linewidth=1)

# Add country borders to plot
ax6.add_feature(country_borders, edgecolor='k', linewidth=1)

# Plot Title
plt.title('250mb Heights and Jet Streaks',loc='left',fontsize=16)
plt.title(' {0:%d %B %Y %H:%MZ}'.format(time_final),loc='right',fontsize=16)


                                        # Heights
#---------------------------------------------------------------------------------------------------

MIN = hght_250.min()
MAX = hght_250.max()

print hght_250.min(),hght_250.max()
hght_250 = ndimage.gaussian_filter(hght_250, sigma=3, order=0) * units.meter

clev250 = np.arange(MIN, MAX, 80)
cs = ax6.contour(lon, lat, hght_250.m, clev250, colors='black', linewidths=2.0,
                linestyles='solid', transform=ccrs.PlateCarree())
#plt.clabel(cs, fontsize=10, inline=1, inline_spacing=10, fmt='%i',
#           rightside_up=True, use_clabeltext=True)

                                        # Winds
#---------------------------------------------------------------------------------------------------
#lon_slice = slice(None, None, 7)
#lat_slice = slice(None, None, 7)
#ax4.barbs(lon[lon_slice], lat[lat_slice],
#         u_250[lon_slice, lat_slice].magnitude,
#         v_250[lon_slice, lat_slice].magnitude,
#         transform=ccrs.PlateCarree(), zorder=2)

wspd250 = mpcalc.get_wind_speed(u_250, v_250)
#print wspd250.min()
clevsped250 = np.arange(50, 100, 1)
cf = ax6.contourf(lon, lat, wspd250, clevsped250, cmap="gist_ncar", transform=ccrs.PlateCarree())
#cbar = plt.colorbar(cf, cax=cax, orientation='horizontal', extend='max', extendrect=True,pad=0.2)
cbaxes = fig.add_axes(colorbar_axis) 
cbar = plt.colorbar(cf, orientation='horizontal',cax=cbaxes)

ax6.set_extent(extent, datacrs)


#---------------------------------------------------------------------------------------------------
#---------------------------------------------------------------------------------------------------

fig.savefig(im_save_path+"250mb_Heights_Winds_"+file_time+".png",
            bbox_inches='tight',dpi=120)
print 'done.'

In [ ]:
os.chdir(im_save_path)
os.system("open .")

# --------------------------------------------------------------------------------------

# <center>End of Daily Maps</center>

# --------------------------------------------------------------------------------------

# We can look further in to the servers we accessed for the raw data

https://doi.org/10.6084/m9.figshare.5244637.v1

# We can access archived data as well for case studies

# Try and get forecasted data, but not working